# Asset Manager Practice Project 01

First, you should implement from scratch, without having a look at the reference implementation, the Mean-Variance Portfolio (MVP) model as a Python class. You should include optimization methods to derive the minimum variance portfolio as well as the maximum Sharpe portfolio.

### Common Imports

In [4]:
import math
import numpy as np
import pandas as pd
from pylab import plt
from scipy.optimize import minimize
plt.style.use('seaborn-v0_8')
pd.set_option("display.precision", 5)
np.set_printoptions(suppress=True,
        formatter={'float': lambda x: f'{x:.4f}'})

In [5]:
import warnings
warnings.filterwarnings('ignore') # I get a RuntimeWarning: Mean of empty slice. etc

### Data

In [153]:
url = 'http://www.muschamp.ca/OffSite/nov25eod.csv'

In [155]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [157]:
raw.head()

,AAPL,NVDA,JPM,SPY,GLD,TLT,EURUSD,BTC-USD
Date,,,,,,,,
2015-11-30,26.6730,0.7738,50.8773,176.3432,101.92,93.0004,1.0632,377.32101
2015-12-01,26.4565,0.7989,51.5869,178.0247,102.28,94.2563,1.0613,362.48801
2015-12-02,26.2175,0.7928,50.8620,176.2071,100.69,94.2869,1.0938,359.18701
2015-12-03,25.9740,0.7911,50.2058,173.7406,101.76,91.7239,1.0876,361.04599
2015-12-07,26.6685,0.8077,51.1214,176.0559,102.67,93.4275,1.0891,395.53601


In [159]:
rets = np.log(raw / raw.shift(1)).dropna()

In [161]:
rets.tail()

,AAPL,NVDA,JPM,SPY,GLD,TLT,EURUSD,BTC-USD
Date,,,,,,,,
2025-11-21,0.01949,-0.00979,-0.00121,0.00991,-0.00155,0.00302,-0.00182,-0.01795
2025-11-24,0.01619,0.02031,-0.00007,0.01461,0.01572,0.00568,0.00061,0.03669
2025-11-25,0.00380,-0.02625,0.01664,0.00936,-0.00032,0.00255,0.00390,-0.01058
2025-11-26,0.00209,0.01363,0.01520,0.00688,0.00797,0.00442,0.00319,0.03572
2025-11-28,0.00467,-0.01825,0.01753,0.00544,0.01235,-0.00476,-0.00009,0.00442


In [163]:
rets.head()

,AAPL,NVDA,JPM,SPY,GLD,TLT,EURUSD,BTC-USD
Date,,,,,,,,
2015-12-01,-0.00815,0.03192,0.01385,0.00949,0.00353,0.01341,-0.00179,-0.04010
2015-12-02,-0.00907,-0.00766,-0.01415,-0.01026,-0.01567,0.00032,0.03016,-0.00915
2015-12-03,-0.00933,-0.00215,-0.01299,-0.01410,0.01057,-0.02756,-0.00568,0.00516
2015-12-07,0.02639,0.02077,0.01807,0.01324,0.00890,0.01840,0.00138,0.09124
2015-12-08,-0.00042,0.01340,-0.01564,-0.00675,0.00165,0.00049,0.01214,0.04939


In [165]:
rets.iloc[0:2]

,AAPL,NVDA,JPM,SPY,GLD,TLT,EURUSD,BTC-USD
Date,,,,,,,,
2015-12-01,-0.00815,0.03192,0.01385,0.00949,0.00353,0.01341,-0.00179,-0.04010
2015-12-02,-0.00907,-0.00766,-0.01415,-0.01026,-0.01567,0.00032,0.03016,-0.00915


In [167]:
universe = rets.columns[:]

In [169]:
universe

Index(['AAPL', 'NVDA', 'JPM', 'SPY', 'GLD', 'TLT', 'EURUSD', 'BTC-USD'], dtype='object')

## MVP Class

In [172]:
class MVPortfolio:
    def __init__(self,
                 market=rets,
                 holdings=None,
                 position_size_range=None
                ):
        self.market=market
        if holdings is None:
            self.holdings = self.market.columns[:]
        else:
            self.holdings = holdings
        self.our_returns = self.market[self.holdings]
        self.noa = len(self.holdings)
        self.set_bnds(position_size_range)
        self.cons = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
        self.equal_weights = self.noa * [1 / self.noa]

    def set_bnds(self, position_size_range=None):
        if position_size_range is None:
            self.bnds = tuple((0,1) for x in range(self.noa))
        else:
            # should check this is the right size and data type
            self.bnds = position_size_range

    @staticmethod        
    def annualized_return(rets, weights):
        return np.dot(rets.mean(), weights) * 252  # annualized

    @staticmethod
    def annualized_volatility(rets, weights):
        return math.sqrt(np.dot(weights, np.dot(rets.cov() * 252 , weights)))

    @classmethod
    def sharpe_ratio(cls, rets, weights):
        return cls.annualized_return(rets, weights) / cls.annualized_volatility(rets, weights)

    def minimum_risk_portfolio(self):
        opt = minimize(lambda weights: MVPortfolio.annualized_volatility(self.our_returns, weights),
                       self.equal_weights, 
                       bounds=self.bnds, 
                       constraints=self.cons)
        return opt['x'] # should I return just opt, this seems to be the answer we want

    def maximum_sharpe_portfolio(self):
        opt = minimize(lambda weights: -MVPortfolio.sharpe_ratio(self.our_returns, weights),
                       self.equal_weights,
                       bounds=self.bnds,
                       constraints=self.cons)
        return opt['x']

In [174]:
symbols = ['AAPL', 'NVDA', 'JPM']
portfolio = MVPortfolio(holdings=symbols)

In [176]:
portfolio.market

,AAPL,NVDA,JPM,SPY,GLD,TLT,EURUSD,BTC-USD
Date,,,,,,,,
2015-12-01,-0.00815,0.03192,0.01385,0.00949,0.00353,0.01341,-0.00179,-0.04010
2015-12-02,-0.00907,-0.00766,-0.01415,-0.01026,-0.01567,0.00032,0.03016,-0.00915
2015-12-03,-0.00933,-0.00215,-0.01299,-0.01410,0.01057,-0.02756,-0.00568,0.00516
2015-12-07,0.02639,0.02077,0.01807,0.01324,0.00890,0.01840,0.00138,0.09124
2015-12-08,-0.00042,0.01340,-0.01564,-0.00675,0.00165,0.00049,0.01214,0.04939
...,...,...,...,...,...,...,...,...
2025-11-21,0.01949,-0.00979,-0.00121,0.00991,-0.00155,0.00302,-0.00182,-0.01795
2025-11-24,0.01619,0.02031,-0.00007,0.01461,0.01572,0.00568,0.00061,0.03669
2025-11-25,0.00380,-0.02625,0.01664,0.00936,-0.00032,0.00255,0.00390,-0.01058


In [178]:
portfolio.holdings

['AAPL', 'NVDA', 'JPM']

In [180]:
portfolio.our_returns

,AAPL,NVDA,JPM
Date,,,
2015-12-01,-0.00815,0.03192,0.01385
2015-12-02,-0.00907,-0.00766,-0.01415
2015-12-03,-0.00933,-0.00215,-0.01299
2015-12-07,0.02639,0.02077,0.01807
2015-12-08,-0.00042,0.01340,-0.01564
...,...,...,...
2025-11-21,0.01949,-0.00979,-0.00121
2025-11-24,0.01619,0.02031,-0.00007
2025-11-25,0.00380,-0.02625,0.01664


In [182]:
portfolio.noa

3

In [184]:
portfolio.bnds

((0, 1), (0, 1), (0, 1))

In [186]:
# You can set the bounds individually, but I don't test you pass in the correct number of bnds
new_bnds = tuple((.05,.95) for x in range(portfolio.noa))
new_bnds

((0.05, 0.95), (0.05, 0.95), (0.05, 0.95))

In [188]:
portfolio.set_bnds(new_bnds)

In [190]:
portfolio.bnds

((0.05, 0.95), (0.05, 0.95), (0.05, 0.95))

In [192]:
portfolio.equal_weights

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

In [194]:
portfolio.annualized_return(portfolio.our_returns, portfolio.equal_weights)

0.36608495673940034

In [196]:
portfolio.annualized_volatility(portfolio.our_returns, portfolio.equal_weights)

0.3000372581279322

In [198]:
MVPortfolio.sharpe_ratio(portfolio.our_returns, portfolio.equal_weights)

1.2201316563935076

In [200]:
minimum_risk_weights = portfolio.minimum_risk_portfolio()

In [202]:
minimum_risk_weights

array([0.3912, 0.0500, 0.5588])

In [204]:
maximum_sharpe_weights = portfolio.maximum_sharpe_portfolio()

In [206]:
maximum_sharpe_weights

array([0.2186, 0.4671, 0.3143])

In [208]:
MVPortfolio.annualized_return(portfolio.our_returns, maximum_sharpe_weights)

0.41447269681256826

Second, you should implement from scratch as a Python class a
rolling backtesting approach for the Mean-Variance Portfolio model.
This class can inherit from the first class. The idea is to allow
the user to define the look-back period (say, 1 or 2 years) from
which data is used to derive the MVP optimal portfolio. Then the
optimal portfolio's performance for the next period (e.g. one week
when working with weekly data) is calculated. The weekly
out-of-sample performances shall be aggregated and presented at the
end as the total performance over the total out-of-sample
backtesting period (for example, 52 weeks = 1 year). The performance
shall also be presented as an annualized return.

## Backtesting Class

In [212]:
class BacktestedPortfolio(MVPortfolio):
    def __init__(self, 
                 market, 
                 holdings, 
                 position_size_range, 
                 look_back_years):
        super().__init__(market, holdings, position_size_range)
        self.look_back_years = look_back_years
        self.set_initial_optimal_weights()

    def set_initial_optimal_weights(self):
        period_returns = self.our_returns.iloc[0:self.look_back_years * 252]
        print(period_returns)
        self.optimal_weights = minimize(lambda weights: -BacktestedPortfolio.sharpe_ratio(period_returns, weights),
                                       portfolio.equal_weights,
                                       bounds=self.bnds,
                                       constraints=self.cons)['x']

    def out_of_sample_annualized_returns(self, trading_day_steps):
        if trading_day_steps is None:
            step_size = 252
        else:
            step_size = trading_day_steps
        # print(step_size)
        double_step = step_size * 2
        res = pd.DataFrame()
        optimal_weights = {}
        period = 1
        ow = self.optimal_weights
        for numeric_index in range(self.look_back_years * 252, len(self.our_returns) - step_size, step_size):
            
            # print(ow)
            optimal_weights[period] = ow
            period_returns = self.our_returns.iloc[numeric_index:numeric_index+step_size]
            # print(numeric_index)
            # print(period_returns)
            epv = BacktestedPortfolio.annualized_volatility(period_returns, ow)
            epr = BacktestedPortfolio.annualized_return(period_returns, ow)
            esr = epr / epv
            next_periods_returns = self.our_returns.iloc[numeric_index+step_size:numeric_index+double_step]
            rpv = BacktestedPortfolio.annualized_volatility(next_periods_returns, ow)
            rpr = BacktestedPortfolio.annualized_return(next_periods_returns, ow)
            rsr = rpr / rpv
            res = pd.concat((res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                           'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                          index=[period])))
            period += 1
            if step_size < (self.look_back_years * 252):
                rolling_start_index = (numeric_index + step_size) - (self.look_back_years * 252)
                rolling_returns = self.our_returns.iloc[rolling_start_index:numeric_index+step_size]
            else: 
                rolling_start_index = numeric_index + step_size
                rolling_returns = self.our_returns.iloc[rolling_start_index:rolling_start_index+step_size]
            # print(rolling_start_index)
            # print(rolling_returns)
            ow = minimize(lambda weights: -BacktestedPortfolio.sharpe_ratio(rolling_returns, weights),
                          portfolio.equal_weights,
                          bounds=self.bnds,
                          constraints=self.cons)['x']
        return optimal_weights, res

    def weekly_out_of_sample_annualized_returns(self):
        return self.out_of_sample_annualized_returns(5) # Five trading days is a week

    def monthly_out_of_sample_annualized_returns(self):
        return self.out_of_sample_annualized_returns(21) # 21 trading days is a month

    def quarterly_out_of_sample_annualized_returns(self):
        return self.out_of_sample_annualized_returns(63) # 63 trading days is a quarter

    def yearly_out_of_sample_annualized_returns(self):
        return self.out_of_sample_annualized_returns(252) # 252 trading days is a year

In [214]:
portfolio = BacktestedPortfolio(holdings=symbols, look_back_years=1, market=rets, position_size_range=new_bnds)

               AAPL     NVDA      JPM
Date                                 
2015-12-01 -0.00815  0.03192  0.01385
2015-12-02 -0.00907 -0.00766 -0.01415
2015-12-03 -0.00933 -0.00215 -0.01299
2015-12-07  0.02639  0.02077  0.01807
2015-12-08 -0.00042  0.01340 -0.01564
...             ...      ...      ...
2017-02-27  0.00293  0.03828 -0.00771
2017-02-28  0.00044 -0.02848  0.00210
2017-03-01  0.02023  0.01286  0.03236
2017-03-02 -0.00595 -0.03758 -0.01572
2017-03-06  0.00273 -0.01353 -0.00239

[252 rows x 3 columns]


In [216]:
portfolio.look_back_years

1

In [218]:
portfolio.optimal_weights

array([0.0500, 0.6690, 0.2810])

In [220]:
portfolio.our_returns[portfolio.look_back_years * 252:]

,AAPL,NVDA,JPM
Date,,,
2017-03-07,0.00129,0.01090,-0.00556
2017-03-08,-0.00373,-0.00185,-0.00219
2017-03-09,-0.00231,-0.00021,0.00394
2017-03-13,0.00374,0.03307,-0.00230
2017-03-14,-0.00151,-0.00072,0.00164
...,...,...,...
2025-11-21,0.01949,-0.00979,-0.00121
2025-11-24,0.01619,0.02031,-0.00007
2025-11-25,0.00380,-0.02625,0.01664


In [222]:
len(portfolio.our_returns) - 252

1950

In [224]:
portfolio.annualized_return(portfolio.our_returns, portfolio.optimal_weights)

0.48777816720528816

In [226]:
weights, our_results = portfolio.weekly_out_of_sample_annualized_returns()

In [227]:
# weights

In [230]:
our_results

,epv,epr,esr,rpv,rpr,rsr
1,0.15283,1.31943,8.63317,0.35938,1.43688,3.99821
2,0.34937,1.29082,3.69471,0.13235,0.70407,5.31981
3,0.13185,0.69247,5.25190,0.33132,-4.21180,-12.71209
4,0.26455,-3.58100,-13.53628,0.23941,0.19863,0.82964
5,0.20494,0.00578,0.02818,0.14414,1.68065,11.65977
...,...,...,...,...,...,...
385,0.19652,-0.48642,-2.47520,0.10746,2.62230,24.40358
386,0.10746,2.62230,24.40358,0.07134,0.37351,5.23557
387,0.06601,0.21672,3.28335,0.30013,-0.54739,-1.82383
388,0.29582,-0.55826,-1.88719,0.19699,-1.80474,-9.16144


In [232]:
weights, our_results = portfolio.monthly_out_of_sample_annualized_returns()

In [234]:
# weights

In [236]:
our_results

,epv,epr,esr,rpv,rpr,rsr
1,0.27530,-0.16909,-0.61420,0.49877,2.09643,4.20322
2,0.33382,1.56285,4.68178,0.26382,0.77644,2.94303
3,0.25873,0.75192,2.90613,0.22475,0.90022,4.00546
4,0.18967,0.88998,4.69228,0.19921,0.04686,0.23523
5,0.19594,0.13993,0.71415,0.19719,0.73664,3.73571
...,...,...,...,...,...,...
88,0.15552,1.21657,7.82278,0.18869,0.02883,0.15277
89,0.18869,0.02883,0.15277,0.12793,0.53008,4.14333
90,0.12719,0.42060,3.30690,0.15118,0.41044,2.71491
91,0.15297,0.36030,2.35529,0.20405,0.17384,0.85198


In [238]:
weights, our_results = portfolio.quarterly_out_of_sample_annualized_returns()

In [240]:
weights

{1: array([0.0500, 0.6690, 0.2810]),
 2: array([0.2185, 0.3924, 0.3892]),
 3: array([0.3583, 0.1546, 0.4871]),
 4: array([0.2321, 0.1774, 0.5905]),
 5: array([0.2542, 0.6958, 0.0500]),
 6: array([0.5031, 0.1130, 0.3839]),
 7: array([0.0500, 0.0500, 0.9000]),
 8: array([0.9000, 0.0500, 0.0500]),
 9: array([0.3601, 0.0500, 0.5899]),
 10: array([0.3996, 0.0500, 0.5504]),
 11: array([0.7770, 0.1730, 0.0500]),
 12: array([0.7034, 0.2466, 0.0500]),
 13: array([0.4748, 0.4752, 0.0500]),
 14: array([0.2156, 0.7344, 0.0500]),
 15: array([0.1631, 0.3860, 0.4509]),
 16: array([0.0500, 0.2945, 0.6555]),
 17: array([0.2210, 0.2893, 0.4898]),
 18: array([0.3562, 0.5938, 0.0500]),
 19: array([0.9000, 0.0500, 0.0500]),
 20: array([0.9000, 0.0500, 0.0500]),
 21: array([0.0500, 0.0500, 0.9000]),
 22: array([0.0500, 0.9000, 0.0500]),
 23: array([0.0500, 0.4335, 0.5165]),
 24: array([0.0500, 0.6517, 0.2983]),
 25: array([0.0500, 0.4742, 0.4758]),
 26: array([0.0500, 0.3118, 0.6382]),
 27: array([0.0639, 0

In [242]:
our_results

,epv,epr,esr,rpv,rpr,rsr
1,0.37519,1.14658,3.05598,0.28806,0.86285,2.99534
2,0.20915,0.69251,3.31108,0.28752,0.46187,1.60640
3,0.24454,0.39271,1.60591,0.23274,0.28698,1.23304
4,0.23725,0.19928,0.83994,0.16575,0.29547,1.78264
5,0.24214,0.17188,0.70982,0.76375,-2.19639,-2.87580
6,0.36657,-1.11571,-3.04361,0.23495,0.49642,2.11289
7,0.20135,0.32989,1.63838,0.24353,0.28947,1.18865
8,0.30504,0.78402,2.57025,0.20106,1.13505,5.64520
9,0.17220,0.85594,4.97062,0.66100,-0.75536,-1.14276
10,0.65447,-0.71047,-1.08557,0.32155,0.90120,2.80264


In [244]:
weights, our_results = portfolio.yearly_out_of_sample_annualized_returns()

In [246]:
weights

{1: array([0.0500, 0.6690, 0.2810]),
 2: array([0.3601, 0.0500, 0.5899]),
 3: array([0.4748, 0.4752, 0.0500]),
 4: array([0.2210, 0.2893, 0.4898]),
 5: array([0.0500, 0.0500, 0.9000]),
 6: array([0.0500, 0.4742, 0.4758]),
 7: array([0.4339, 0.0500, 0.5161])}

In [248]:
our_results

,epv,epr,esr,rpv,rpr,rsr
1,0.33931,0.74039,2.18204,0.45935,-0.19840,-0.43191
2,0.23872,0.11107,0.46527,0.40966,0.38177,0.93192
3,0.46300,0.87733,1.89490,0.28355,0.55504,1.95745
4,0.19882,0.44978,2.26219,0.34993,-0.24582,-0.70248
5,0.29631,-0.15702,-0.52991,0.20192,0.41622,2.06131
6,0.30326,0.91444,3.01538,0.30479,0.33829,1.10992
7,0.17497,0.30707,1.75502,0.26817,0.29272,1.09156
